In [ ]:
#!python -m spacy download el_core_news_sm

In [1]:
import spacy
import el_core_news_sm
import string
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from spacy.lang.el import GreekLemmatizer

In [3]:
from spacy.lang.el import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

In [4]:
nlp = el_core_news_sm.load()

In [5]:
lemmatizer = GreekLemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

In [6]:
def loadStopWords():
    sWords = open('stopWords.txt','r',encoding='utf-8')
    sw = set(sWords.read().split('\n'))
    sWords.close()
    return sw

In [7]:
#sw = nlp.Defaults.stop_words
#sw = sw|{'εχω','απο','ωστε'}
sw = loadStopWords()
def remove_ton(text):
    diction = {'ά':'α','έ':'ε','ί':'ι','ό':'ο','ώ':'ω','ύ':'υ'}
    for key in diction.keys():
        text = text.replace(key, diction[key])
    return text   
def clean_text(text):
     #text to string
    text = str(text).lower()
   # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # lower text
    text = [remove_ton(x) for x in text]
    # remove stop words
    text = [x for x in text if x not in sw]
 
    #remove quotes
    text = [x.replace('quot;','').replace('&quot','') for x in text if x not in ['quot','amp']]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove amp & quot
    text = [x for x in text if x not in ['quot','amp']]
    # remove words with only one letter
    text = " ".join([t for t in text if len(t) > 1])
    # lemmatize text
    text = " ".join([lemmatizer(t.text,t.pos_)[0] for t in nlp(text)])
   
    return(text)

In [8]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '6ed9d167-b2e6-41b8-9500-35e6df64d9dc'
resource_group = 'MLRG'
workspace_name = 'erbbimlws'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='vBankingComments')
df = dataset.to_pandas_dataframe()

In [9]:
df.head()

,CON_ROW_ID,CON_COMMENTS,Source
0,65444303.0,ανανεωση γνωματευσης οκ,Contact Type
1,63843091.0,ενημερωση για μη υποβολη αιτήματος ΠΡΟΕΞΟΦΛΗΣΗ...,Contact Type
2,65048084.0,θα παει στο καταστημα για εκδοση Business debit,Contact Type
3,64958601.0,ΖΗΤΗΣΕ ΑΠΟΔΕΙΚΤΙΚΟ ΤΕΚΜΗΡΙΩΣΗΣ ΕΤΑΙΡΙΚΟΥ ΛΟΓΑΡ...,Contact Type
4,64878281.0,επιταγη,Contact Type


In [11]:
df['tokenized'] = df['CON_COMMENTS'].apply(clean_text)

In [12]:
df.head(1000)

,CON_ROW_ID,CON_COMMENTS,Source,tokenized
0,65444303.0,ανανεωση γνωματευσης οκ,Contact Type,ανανεωση γνωματευση οκ
1,63843091.0,ενημερωση για μη υποβολη αιτήματος ΠΡΟΕΞΟΦΛΗΣΗ...,Contact Type,ενημερωση υποβολη αιτήμα προεξοφληση ατοκος δο...
2,65048084.0,θα παει στο καταστημα για εκδοση Business debit,Contact Type,παει καταστημας εκδοση business debit
3,64958601.0,ΖΗΤΗΣΕ ΑΠΟΔΕΙΚΤΙΚΟ ΤΕΚΜΗΡΙΩΣΗΣ ΕΤΑΙΡΙΚΟΥ ΛΟΓΑΡ...,Contact Type,ζητησε αποδεικτικος τεκμηριωση εταιρικος λογαρ...
4,64878281.0,επιταγη,Contact Type,επιταγη
...,...,...,...,...
995,64792612.0,ΓΙΑ ΚΑΡΝΕ ΕΠΙΤΑΓΩΝ,Contact Type,καρνε επιταγο
996,65731484.0,έγινε η βιντεοκλήση και ενημερώθηκε για τις δυ...,Contact Type,εγινε βιντεοκλήση ενημερωθηκε δυνατοτητα
997,65735510.0,ζητησε πληροφοριες για να καταθεσει ποσο εκτος...,Contact Type,ζητησε πληροφορια καταθεσω ποσο ωραριου κατ / των
998,66657265.0,v-call για βεβαίωση ΜΜΥ για προγραμμα ΕΣΠΑ,Contact Type,v-call βεβαιωση μμυ προγραμμο εσπας


In [13]:
df = df.fillna('N/A')

In [17]:
tfidf = TfidfVectorizer(min_df = 100,ngram_range = (1,2))

In [18]:
tfidf_result = tfidf.fit_transform(df['tokenized']).toarray()

In [19]:
tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())

In [20]:
tfidf_df.columns = [str(x) for x in tfidf_df.columns]

In [22]:
df_f = pd.concat([df[['CON_ROW_ID']],tfidf_df],axis=1).melt(id_vars=['CON_ROW_ID'],value_vars = tfidf_df.columns).dropna()

In [23]:
df_f = df_f[df_f['value']>0]

In [29]:
#!pip install openpyxl
#import openpyxl

In [28]:
df_f[['CON_ROW_ID','variable']].to_excel('vBanking_tokens.xlsx',index = False)
#df_f[df_f['value']>0].to_excel('D://Downloads//comments_tokens.xlsx')
#df.to_excel('D://Downloads//comments_cleaned.xlsx')